<a href="https://colab.research.google.com/github/Leopalds/metodo-rigidez/blob/main/MAC023-Trabalho2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
## includes
import numpy as np
import math

## Definição das Classes de Barras

In [48]:
class BarraPorticoPlano:
    def __init__(self, id, L, A, I, E, angulo):
      self.id = id # identificador da barra
      self.L = L # comprimento da barra (m)
      self.A = A # area da seção transversal (m²)
      self.I = I # momento de inércia da seção (m⁴)
      self.E = E # módulo de elasticidade do material (Pa)
      self.angulo = math.radians(angulo) # angulo de inclinação da barra em relação ao eixo global, recebido em graus e convertido para radianos

    def rigidezAxial(self):
      # rigidez axial da barra (EI/L)
      return (self.E * self.A) / self.L

    def rigidezFlexional(self):
        # rigidez flexional da barra (EI/L³)
        return (self.E * self.I) / (self.L ** 3)

    def matrizRigidezLocal(self):
      # matriz de rigidez local (tabelada)
      L = self.L
      E = self.E
      A = self.A
      I = self.I
      k = [
          [ E*A/L,  0,          0,         -E*A/L,  0,         0       ],
          [ 0,      12*E*I/L**3, 6*E*I/L**2, 0,     -12*E*I/L**3, 6*E*I/L**2 ],
          [ 0,      6*E*I/L**2,  4*E*I/L,   0,     -6*E*I/L**2,  2*E*I/L   ],
          [-E*A/L,  0,          0,          E*A/L,  0,         0       ],
          [ 0,     -12*E*I/L**3, -6*E*I/L**2, 0,      12*E*I/L**3, -6*E*I/L**2],
          [ 0,      6*E*I/L**2,  2*E*I/L,   0,     -6*E*I/L**2,  4*E*I/L   ]
      ]
      return k
    def matrizRotacao(self):
      # matriz de rotação local
      cx = np.cos(self.angulo)
      cy = np.sin(self.angulo)

      R = np.array([
          [ cx,  cy,  0,  0,  0,  0],
          [-cy,  cx,  0,  0,  0,  0],
          [ 0,  0,  1,  0,  0,  0],
          [ 0,  0,  0,  cx,  cy,  0],
          [ 0,  0,  0, -cy,  cx,  0],
          [ 0,  0,  0,  0,  0,  1]
      ])

      return R

In [49]:
class BarraTrelicaPlana:
    def __init__(self, id, coordNoInic, coordNoFim, A, E):
      self.id = id
      self.coordNoInic = np.array(coordNoInic)
      self.coordNoFim = np.array(coordNoFim)
      self.A = A
      self.E = E
      self.L = self.calcular_L()
      self.angulo = self.calcular_angulo()

    def calcularL(self):
      return np.linalg.norm(self.coordNoFim - self.coordNoInic) # distância euclidiana

    def calcularAngulo(self):
      distX = self.coordNoFim[0] - self.coordNoInic[0]
      distY = self.coordNoFim[1] - self.coordNoInic[1]
      return np.arctan2(distY, distX) # ângulo em rad

    def matrizRotacao(self):
      cx = np.cos(self.angulo)
      cy = np.sin(self.angulo)

      R = np.array([
          [ cx,  cy,  0,  0],
          [-cy,  cx,  0,  0],
          [ 0,  0,  cx,  cy],
          [ 0,  0, -cy,  cx]
      ])
      return R

    def matrizRigidezLocal(self):
      EA_L = (self.E * self.A) / self.L

      kLocal = np.array([
          [ EA_L,  0, -EA_L,  0],
          [ 0,  0,  0,  0],
          [-EA_L,  0,  EA_L,  0],
          [ 0,  0,  0,  0]
      ]) * EA_L

      return kLocal


In [50]:
class BarraVigaPlana:
    def __init__(self, id, coordNoInic, coordNoFim, A, I, E):
      self.id = id
      self.coordNoInic = np.array(coordNoInic)
      self.coordNoFim = np.array(coordNoFim)
      self.A = A
      self.I = I
      self.E = E
      self.L = self.calcularComprimento()
      self.angulo = self.calcularAngulo()

    def calcularComprimento(self):
      return np.linalg.norm(self.coordNoFim - self.coordNoInic)

    def calcularAngulo(self):
      distX = self.coordNoFim[0] - self.coordNoInic[0]
      distY = self.coordNoFim[1] - self.coordNoInic[1]
      return np.arctan2(distY, distX)

    def matrizRotacao(self):
      cx = np.cos(self.angulo)
      cy = np.sin(self.angulo)

      R = np.array([
          [ cx,  cy,  0,  0],
          [-cy,  cx,  0,  0],
          [ 0,  0,   cx,  cy],
          [ 0,  0,  -cy,  cx],
      ])
      return R

    def matrizRigidezLocal(self):
      L = self.L
      E = self.E
      I = self.I

      kLocal = np.array([
          [ 12*E*I/L**3,  6*E*I/L**2, -12*E*I/L**3,  6*E*I/L**2],
          [ 6*E*I/L**2,   4*E*I/L,    -6*E*I/L**2,   2*E*I/L  ],
          [-12*E*I/L**3, -6*E*I/L**2,  12*E*I/L**3, -6*E*I/L**2],
          [ 6*E*I/L**2,   2*E*I/L,    -6*E*I/L**2,   4*E*I/L  ]
      ])

      return kLocal

    def imprimirBarra(self):
        print(f"ID: {self.id}, NoI: {self.coordNoInic}, NoF: {self.coordNoFim}, L: {self.L:.2f}, Ângulo: {self.angulo:.2f}, A: {self.A:.2f}, I: {self.I:.2e}, E: {self.E:.2e}")

## Entrada de dados

In [51]:
import json

def read_data_json(filename):
    # Abre e carrega o arquivo JSON
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Extração dos parâmetros principais, garantindo que existam no JSON
    try:
        nno        = int(data["nno"])
        nel        = int(data["nel"])
        nnoel      = int(data["nnoel"])
        ndofn      = int(data["ndofn"])
        ntm        = int(data["ntm"])
        nts        = int(data["nts"])
        typestruct = int(data["typestruct"])
    except KeyError as e:
        raise Exception(f"Atenção: a chave {e} não foi encontrada no JSON")

    # Validação da lista de coordenadas (no JSON a chave é 'coord')
    coord_list = data.get("coord")
    if coord_list is None:
        raise Exception("Atenção: chave 'coord' não encontrada no JSON")
    if len(coord_list) != nno:
        raise Exception("Atenção: número de nós em 'coord' diferente de nno.")

    # Validação da conectividade dos elementos
    ndn_list = data.get("ndn")
    if ndn_list is None:
        raise Exception("Atenção: chave 'ndn' não encontrada no JSON")
    if len(ndn_list) != nel:
        raise Exception("Atenção: número de elementos em 'ndn' diferente de nel.")

    # Validação dos tipos de material
    intm_list = data.get("intm")
    if intm_list is None:
        raise Exception("Atenção: chave 'intm' não encontrada no JSON")
    if len(intm_list) != nel:
        raise Exception("Atenção: lista 'intm' com tamanho diferente de nel.")

    # Validação dos tipos de seção
    ints_list = data.get("ints")
    if ints_list is None:
        raise Exception("Atenção: chave 'ints' não encontrada no JSON")
    if len(ints_list) != nel:
        raise Exception("Atenção: lista 'ints' com tamanho diferente de nel.")

    # Validação das propriedades do material
    E_list = data.get("E")
    if E_list is None:
        raise Exception("Atenção: chave 'E' não encontrada no JSON")
    if len(E_list) != ntm:
        raise Exception("Atenção: número de valores de E diferente de ntm.")

    nu_list = data.get("nu")
    if nu_list is None:
        raise Exception("Atenção: chave 'nu' não encontrada no JSON")
    if len(nu_list) != ntm:
        raise Exception("Atenção: número de valores de nu diferente de ntm.")

    rho_list = data.get("rho")
    if rho_list is None:
        raise Exception("Atenção: chave 'rho' não encontrada no JSON")
    if len(rho_list) != ntm:
        raise Exception("Atenção: número de valores de rho diferente de ntm.")

    # Validação das propriedades da seção
    A_list = data.get("A")
    if A_list is None:
        raise Exception("Atenção: chave 'A' não encontrada no JSON")
    if len(A_list) != nts:
        raise Exception("Atenção: número de valores de A diferente de nts.")

    # A inércia pode estar como 'Iz' ou 'I'
    if "Iz" in data:
        Iz_list = data.get("Iz")
    elif "I" in data:
        Iz_list = data.get("I")
    else:
        raise Exception("Atenção: chave 'Iz' ou 'I' não encontrada no JSON")
    if len(Iz_list) != nts:
        raise Exception("Atenção: número de valores de Iz/I diferente de nts.")

    sf_list = data.get("sf")
    if sf_list is None:
        raise Exception("Atenção: chave 'sf' não encontrada no JSON")
    if len(sf_list) != nts:
        raise Exception("Atenção: número de valores de sf diferente de nts.")

    # Cálculo do módulo de elasticidade transversal G para cada material
    G_list = []
    for E_val, nu_val in zip(E_list, nu_list):
        G_list.append(E_val / (2 * (1 + nu_val)))

    # Organiza os dados em um dicionário, padronizando a chave de coordenadas para 'coor'
    new_data = {
        'nno': nno,
        'nel': nel,
        'nnoel': nnoel,
        'ndofn': ndofn,
        'ntm': ntm,
        'nts': nts,
        'typestruct': typestruct,
        'coor': coord_list,  # renomeia 'coord' para 'coor'
        'ndn': ndn_list,
        'intm': intm_list,
        'ints': ints_list,
        'E': E_list,
        'nu': nu_list,
        'rho': rho_list,
        'A': A_list,
        'Iz': Iz_list,
        'sf': sf_list,
        'G': G_list
    }

    return new_data

filename = 'dados_modelo.json'
data = read_data_json(filename)
print(data)


{'nno': 5, 'nel': 4, 'nnoel': 2, 'ndofn': 2, 'ntm': 1, 'nts': 1, 'typestruct': 1, 'coor': [[0, 0], [8, 0], [14, 0], [20, 0], [23, 0]], 'ndn': [[1, 2], [2, 3], [3, 4], [4, 5]], 'intm': [1, 1, 1, 1], 'ints': [1, 1, 1, 1], 'E': [1670000], 'nu': [0.3], 'rho': [1], 'A': [0.18], 'Iz': [0.0054], 'sf': [1], 'G': [642307.6923076923]}


## Instanciando as barras

In [53]:
barras = []
for i in range(data['nel']):
  #Pegando a area da barra
  indiceArea = data['ints'][i]
  area = data['A'][indiceArea - 1]

  # Pegando o Iz
  indiceI = data['intm'][i]
  Iz = data['Iz'][indiceI - 1]

  # Pegando o E
  indiceE = data['intm'][i]
  E = data['E'][indiceE - 1]

  # Pegando as coordenadas dos nos
  indiceNoIncial, indiceNoFinal = data['ndn'][i]
  noInicial = data['coor'][indiceNoIncial - 1]
  noFinal = data['coor'][indiceNoFinal - 1]

  if(data['typestruct'] == 1):
    # Viga
    # print(E)
    barras.append(BarraVigaPlana(i, noInicial, noFinal, area, Iz, E))
    barras[i].imprimirBarra()
  elif(data['typestruct'] == 2):
    # Treliça
    # TODO: Implement this
    raise Exception("NOT IMPLEMENTED")
  elif(data['typestruct'] == 3):
    # Portico
    # TODO: Implement this
    raise Exception("NOT IMPLEMENTED")
  else:
    raise Exception("Tipo desconhecido")

ID: 0, NoI: [0 0], NoF: [8 0], L: 8.00, Ângulo: 0.00, A: 0.18, I: 5.40e-03, E: 1.67e+06
ID: 1, NoI: [8 0], NoF: [14  0], L: 6.00, Ângulo: 0.00, A: 0.18, I: 5.40e-03, E: 1.67e+06
ID: 2, NoI: [14  0], NoF: [20  0], L: 6.00, Ângulo: 0.00, A: 0.18, I: 5.40e-03, E: 1.67e+06
ID: 3, NoI: [20  0], NoF: [23  0], L: 3.00, Ângulo: 0.00, A: 0.18, I: 5.40e-03, E: 1.67e+06


## Calculo da matriz global

### Rotina para montar o vetor LM

In [54]:
def calculo_vetor_LM(numero_elemento, nnoel, ndofn):
  # Calculando o número de dofs por elemento
    edof = nnoel * ndofn
    start = (numero_elemento - 1) * (nnoel - 1) * ndofn

    # Criando o vetor LM
    vetor_LM = np.arange(start + 1, start + edof + 1)
    return vetor_LM

### Rotina para colocar a matriz de rigidez local na global

In [55]:
def assembly(kk, k, vetor_LM):
    # Número de graus de liberdade por elemento
    edof = len(vetor_LM)

    # Montagem da matriz do sistema
    for i in range(edof):
        ii = vetor_LM[i] - 1
        for j in range(edof):
            jj = vetor_LM[j] - 1
            kk[ii, jj] += k[i, j]  # Adiciona o valor de k na matriz do sistema

    return kk

### Montagem da matriz global

In [56]:
matriz_global = np.zeros((data['nno']* data['ndofn'], data['nno']* data['ndofn']))

for (i, barra) in enumerate(barras):
  matrizRotacao = np.array(barra.matrizRotacao())
  matrizRigidezRotacionada = matrizRotacao.T @ barra.matrizRigidezLocal() @ matrizRotacao
  vetor_lm = calculo_vetor_LM(barra.id+1, data['nnoel'], data['ndofn'])
  matriz_global = assembly(matriz_global, matrizRigidezRotacionada, vetor_lm)

# print(np.array2string(matriz_global, precision=2, separator=', ', formatter={'float_kind': lambda x: f"{x:.2f}"}, max_line_width=100))

## Finalizar a entrada de dados

In [57]:
import json

def ler_cargas_struct(filename, num_nodes, num_elements, ndofn):
    """
    Lê e valida os dados de carregamento de um arquivo JSON.

    :param filename: Nome do arquivo JSON com as informações de cargas.
    :param num_nodes: Número total de nós no modelo (para checar limites).
    :param num_elements: Número total de elementos no modelo (para checar limites).
    :param ndofn: Graus de liberdade por nó (para checar o formato de ccno).
    :return: Dicionário com os dados de carregamento (se válido).
    :raises ValueError: Caso alguma inconsistência seja encontrada.
    """

    # 1. Leitura do arquivo JSON
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 2. Extração dos dados principais
    nnc   = data["nnc"]       # Número de nós com cargas concentradas
    nele  = data["nele"]      # Número de elementos carregados
    nnoc  = data["nnoc"]      # Lista com os índices dos nós carregados
    ccno  = data["ccno"]      # Lista com as cargas concentradas [nnc x ndofn]
    ndmbc = data["ndmbc"]     # Lista com os índices dos elementos carregados
    ityl0 = data["itylo"]     # Lista com os dados dos tipos de carga
    nccdesl = data["nccdesl"] # Número de nós com apoio
    nnr = data['nnr']         # Lista com os indíces dos nós com apoio
    itydisp = data['itydisp'] # Lista com qual grau de liberdade está restrito

    # 3. Validações básicas de tamanho
    # 3.1. nnc vs nnoc
    if len(nnoc) != nnc:
        raise ValueError(
            f"Inconsistência: nnc={nnc}, mas len(nnoc)={len(nnoc)}."
        )

    # 3.2. nnc vs ccno
    if len(ccno) != nnc:
        raise ValueError(
            f"Inconsistência: nnc={nnc}, mas len(ccno)={len(ccno)}."
        )
    # Checando se cada linha de ccno tem ndofn colunas
    for i, linha_ccno in enumerate(ccno, start=1):
        if len(linha_ccno) != ndofn:
            raise ValueError(
                f"Em ccno, a linha {i} deveria ter {ndofn} valores, mas tem {len(linha_ccno)}."
            )

    # 3.3. nele vs ndmbc
    if len(ndmbc) != nele:
        raise ValueError(
            f"Inconsistência: nele={nele}, mas len(ndmbc)={len(ndmbc)}."
        )

    # 3.4. nele vs ityl0
    if len(ityl0) != nele:
        raise ValueError(
            f"Inconsistência: nele={nele}, mas len(itylo)={len(ityl0)}."
        )

    if len(nnr) != nccdesl:
      raise ValueError(
            f"Inconsistência: nccdesl={nccdesl}, mas len(nnr)={len(nnr)}."
        )

    if len(itydisp) != nccdesl:
      raise ValueError(
            f"Inconsistência: nccdesl={nccdesl}, mas len(itydisp)={len(itydisp)}."
        )

    # 4. Validando intervalos de nós e elementos
    # 4.1. Índices de nós carregados
    for node_index in nnoc:
        if not (1 <= node_index <= num_nodes):
            raise ValueError(
                f"Nó carregado {node_index} fora do intervalo [1, {num_nodes}]."
            )

    # 4.2. Índices de elementos carregados
    for elem_index in ndmbc:
        if not (1 <= elem_index <= num_elements):
            raise ValueError(
                f"Elemento carregado {elem_index} fora do intervalo [1, {num_elements}]."
            )

    # 4.3. Índices de nós apoiados
    for node_index in nnr:
        if not (1 <= node_index <= num_nodes):
            raise ValueError(
                f"Nó apoiado {node_index} fora do intervalo [1, {num_nodes}]."
            )

    # 4.4. Tamanaho de graus de liberdade dós nos apoiados
    for (i, no_index) in enumerate(itydisp):
      if len(no_index) != ndofn:
        raise ValueError(
              f"Inconsistência: ndofn={ndofn}, mas len(itydisp[{i}])={len(no_index)}."
          )

    # 5. Validação dos tipos de carga
    # Definição dos campos obrigatórios para cada tipo
    # (Ajuste conforme suas necessidades reais de validação)
    tipos_carga = {
        1: ["type", "mag", "pos"],                # Força vertical no vão
        2: ["type", "mag", "pos"],                # Momento no vão
        3: ["type", "mag", "pos"],                # Força horizontal no vão
        4: ["type", "mag", "pos"],                # Torsor no vão
        5: ["type", "mag", "distanciaNo"],        # Forças de mesma magnitude
        6: ["type", "mag"],                       # Carga distribuída (todo vão)
        7: ["type", "mag", "posInicial", "posFinal"],  # Carga distribuída parcial
        8: ["type", "altura", "ladoAltura"]       # Carga triangular
    }

    # 5.1. Checando cada item de ityl0
    for i, item in enumerate(ityl0, start=1):
        load_type = item.get("type")
        if load_type is None:
            raise ValueError(f"Não foi especificado 'type' para a carga {i}.")

        if load_type not in tipos_carga:
            raise ValueError(
                f"Tipo de carga inválido: {load_type}. Deve ser um inteiro de 1 a 8."
            )

        # Checa se os campos obrigatórios existem
        campos_necessarios = tipos_carga[load_type]
        for campo in campos_necessarios:
            if campo not in item:
                raise ValueError(
                    f"Para 'type'={load_type}, o campo '{campo}' é obrigatório. "
                    f"Carga {i} está incompleta."
                )

        # Exemplo extra de validação de formato, se desejado:
        # (Por exemplo, se "pos" deve ser uma lista de 2 valores)
        if "pos" in campos_necessarios:
            pos = item["pos"]
            if not (isinstance(pos, list) and len(pos) == 2):
                raise ValueError(
                    f"Para 'type'={load_type}, 'pos' deve ser uma lista [x, y]."
                )
        if "posInicial" in campos_necessarios:
            if not (isinstance(item["posInicial"], list) and len(item["posInicial"]) == 2):
                raise ValueError(
                    f"Para 'type'={load_type}, 'posInicial' deve ser uma lista [x, y]."
                )
        if "posFinal" in campos_necessarios:
            if not (isinstance(item["posFinal"], list) and len(item["posFinal"]) == 2):
                raise ValueError(
                    f"Para 'type'={load_type}, 'posFinal' deve ser uma lista [x, y]."
                )

    return data

forcas = ler_cargas_struct("cargas.txt", data['nno'], data['nel'], data['ndofn'])
print(forcas)

{'nnc': 1, 'nele': 3, 'nnoc': [5], 'ccno': [[-10, 0]], 'ndmbc': [1, 2, 3], 'itylo': [{'type': 1, 'mag': -8, 'pos': [4, 0]}, {'type': 6, 'mag': -10}, {'type': 6, 'mag': -10}], 'nccdesl': 4, 'nnr': [1, 2, 3, 4], 'itydisp': [[1, 1], [1, 0], [1, 0], [1, 0]]}


In [58]:
def distancia_entre_pontos(ponto1, ponto2):
    return math.sqrt((ponto2[0] - ponto1[0]) ** 2 + (ponto2[1] - ponto1[1]) ** 2)

In [59]:
# agora que temos as forças precisamos montar o vetor de forças para o sistema utilizando os dados de entrada
F = np.zeros(data['nno']* data['ndofn'])

#Forças que atuam nos nós

for (i, forca) in enumerate(forcas['ccno']):
  indiceForca = forcas['nnoc'][i]

  for (j, valor_no_grau_de_lib) in enumerate(forca):
    indiceGrauDeLiberdade = (indiceForca - 1) * data['ndofn'] + j
    F[indiceGrauDeLiberdade] = valor_no_grau_de_lib


# Forças que atuam no elemento
for (i, forca) in enumerate(forcas['itylo']):
  if data['typestruct'] == 2:
    raise Exception("Treliças não tem carregamentos ao longo da barra, apenas nos nós")

  tipo_carga = forca['type']
  elementoId = forcas['ndmbc'][i]
  barra = barras[elementoId - 1]
  L = barra.L           # comprimento barra
  noInicial = barra.coordNoInic
  ids_nos_da_barra = data['ndn'][elementoId-1]

  # força equivalente local
  feq = np.zeros(data['nnoel'] * data['ndofn'])


  if tipo_carga == 1:
        P = forca["mag"]       # Magnitude da força
        posicao = forca["pos"]    # posição

        #encontrando o a
        a = distancia_entre_pontos(noInicial, posicao)
        b = L - a

        # Usando a tabela
        VA = P * b**2 / L**3 * (3*a + b)
        VB = P * a**2 /L**3 * (a + 3*b)
        # Momentos (verifique com a tabela de engastamento perfeito!)
        MA = (P * a* b**2)/L**2
        MB = -(P*a**2*b)/L**2

        if(data['typestruct'] == 1):
          feq[0] = VA
          feq[1] = MA
          feq[2] = VB
          feq[3] = MB
        else:
          feq[1] = VA
          feq[2] = MA
          feq[4] = VB
          feq[5] = MB
  elif tipo_carga == 2:
      M = forca["mag"]
      posicao = forca["pos"]    # posição

      #encontrando o a
      a = distancia_entre_pontos(noInicial, posicao)
      b = L - a
      MA = M*b/L**2 * (2*a - b)
      MB = M*a/L**2 * (2*b - a)
      VA = (6*M*a*b) / L**3
      VB = - VA
      if(data['typestruct'] == 1):
          feq[0] = VA
          feq[1] = MA
          feq[2] = VB
          feq[3] = MB
      else:
        feq[1] = VA
        feq[2] = MA
        feq[4] = VB
        feq[5] = MB

  elif tipo_carga == 3:
      P = forca["mag"]
      posicao = forca["pos"]    # posição

      #encontrando o a
      a = distancia_entre_pontos(noInicial, posicao)
      b = L - a
      VA = P*b/L
      VB = P*a/L
      if(data['typestruct'] == 1):
          feq[0] = VA
          feq[2] = VB
      else:
        feq[1] = VA
        feq[4] = VB

  elif tipo_carga == 4:
      raise Exception("Não tratamos problema com torsor")

  elif tipo_carga == 5:
      P = forca["mag"]
      a = forca["distanciaNo"] #distancia a

      VA = VB = P
      MA = P*a/L*(L-a)
      MB = - MA
      if(data['typestruct'] == 1):
          feq[0] = VA
          feq[1] = MA
          feq[2] = VB
          feq[3] = MB
      else:
        feq[1] = VA
        feq[2] = MA
        feq[4] = VB
        feq[5] = MB

  elif tipo_carga == 6:
      # Carga distribuída q ao longo de todo o vão
      q = forca["mag"]
      VA = q*L/2.0
      MA = q*(L**2)/12.0
      VB = q*L/2.0
      MB = - q*(L**2)/12.0

      if(data['typestruct'] == 1):
          feq[0] = VA
          feq[1] = MA
          feq[2] = VB
          feq[3] = MB
      else:
        feq[0] = 0
        feq[1] = VA
        feq[2] = MA
        feq[3] = 0
        feq[4] = VB
        feq[5] = MB

  elif tipo_carga == 7:
      q = forca["mag"]
      posInicial = forca["posInicial"] #posicaoInicial
      posFinal = forca["posFinal"] #posicaoInicial
      a = distancia_entre_pontos(posInicial, posFinal)

      MA = q*a**2/(12*L**2) * (6*L**2 - 8*a*L + 3*a**2)
      MB = - q * a**3/(12* L**2) * (4*L - 3*a)
      VA = q*a/(2*L**3) * (2* L**3 - 2*a**2*L + a**3)
      VB = q*a**3/(2*L**3) * (2*L - a)

      ## caso o carregamento comece no meio da barra e vá até o final
      if(posInicial != noInicial):
        aux = MA
        MA = MB
        MB = aux
        aux = VA
        VA = VB
        VB = aux

      if(data['typestruct'] == 1):
          feq[0] = VA
          feq[1] = MA
          feq[2] = VB
          feq[3] = MB
      else:
        feq[1] = VA
        feq[2] = MA
        feq[4] = VB
        feq[5] = MB

  elif tipo_carga == 8:
      q = forca["altura"]
      ladoAltura = forca['ladoAltura']
      a = distancia_entre_pontos(posInicial, posFinal)

      MA = q*L**2/30
      MB = - q*L**2/20
      VA = 3*q*L/20
      VB = 7*q*L/20

      ## caso o maior lado do triangulo esteja a esquerda
      if(ladoAltura == 0):
        aux = MA
        MA = MB
        MB = aux
        aux = VA
        VA = VB
        VB = aux

      if(data['typestruct'] == 1):
          feq[0] = VA
          feq[1] = MA
          feq[2] = VB
          feq[3] = MB
      else:
        feq[1] = VA
        feq[2] = MA
        feq[4] = VB
        feq[5] = MB

  else:
      raise Exception(
          f"Tipo de carga {tipo_carga} ainda não implementado na função."
      )

  # se for inputado os graus deliberdade já no eixo global não precisa desse passo
  feq = barra.matrizRotacao() @ feq

  for ii in range(data['nnoel']):
    GDLInicialGlobalDoNo = (ids_nos_da_barra[ii] - 1) * data['ndofn']
    for jj in range(data['ndofn']):
      F[GDLInicialGlobalDoNo + jj] += feq[jj + (ii * data['ndofn'])]

## Tratando os apoios

In [60]:
apoios = np.zeros(forcas['nccdesl'])
restricoes = []

for (i, graus_de_liberdade) in enumerate(forcas['itydisp']) :
  indiceNo = forcas['nnr'][i]

  for(j, restricao) in enumerate(graus_de_liberdade):
    # "remover" a linha e coluna relacionada a esse grau de liberdade
    grauDeLiberdadeGlobal = (indiceNo - 1) * data['ndofn'] + j

    if(restricao == 1):
      restricoes.append(grauDeLiberdadeGlobal)

matriz_global_somente_incognitas = np.delete(matriz_global, restricoes, axis=0)
matriz_global_somente_incognitas = np.delete(matriz_global_somente_incognitas, restricoes, axis=1)
F_somente_incognitas = np.delete(F, restricoes)
print(matriz_global_somente_incognitas, F_somente_incognitas)

[[10521.  3006.     0.     0.     0.]
 [ 3006. 12024.  3006.     0.     0.]
 [    0.  3006. 18036. -6012.  6012.]
 [    0.     0. -6012.  4008. -6012.]
 [    0.     0.  6012. -6012. 12024.]] [-22.   0.  30. -10.   0.]


## Resolução do sistema

In [61]:
deslocamentos = np.linalg.solve(matriz_global_somente_incognitas, F_somente_incognitas)
print(deslocamentos)

[-0.00227693  0.00065055 -0.00032528 -0.01095587 -0.0053153 ]


### Cálcluo das reações de apoio

In [62]:
graus_de_liberdade = np.arange(0, (data['ndofn'] * data['nno']))
graus_de_liberdade_restantes = np.setdiff1d(graus_de_liberdade, np.array(restricoes))

matriz_para_reacoes_apoio = np.delete(matriz_global, graus_de_liberdade_restantes, axis=0)
matriz_para_reacoes_apoio = np.delete(matriz_para_reacoes_apoio, restricoes, axis=1)
F_para_reacoes_apoio = np.delete(F, graus_de_liberdade_restantes)
print(matriz_para_reacoes_apoio, - F_para_reacoes_apoio)

[[  845.4375     0.         0.         0.         0.    ]
 [ 2254.5        0.         0.         0.         0.    ]
 [  657.5625  1503.         0.         0.         0.    ]
 [-1503.         0.      1503.         0.         0.    ]
 [    0.     -1503.      4509.     -4008.      6012.    ]] [ 4.  8. 34. 60. 30.]


$$
\{F_R\} = \{ F_{RL}\} + [K_{RD}]\{d\}
$$

In [63]:
reacoes = - F_para_reacoes_apoio + (matriz_para_reacoes_apoio @  deslocamentos)
reacoes

array([ 2.075     ,  2.86666667, 33.48055556, 62.93333333, 39.51111111])



*   ✅ Instanciar os objetos que o Gustavo criou
  *   ✅Criar a matriz local
*   ✅Criar a matriz global
  *   ✅ rotacionar e colocar a matriz local na global
*   Finalizar a entrada de dados
  *   Apoios
  *   ✅ Forças (geral momento tb) pontuais e carregamentos
*   ✅ Vetor de forças
  *   ✅ Vetor de cargas nodais equivalentes
        *  ✅ Engasmento perfeito com sinal trocado
  *  ✅ Cargas aplicadas diretamente sobre os nos
* ✅ Encontrar as reações de apoio
* Refatorar código
  * Montar relatório
  * Criar funções
  * Utilizar json na primeira leitura também
* Corrigir e testar para outros exemplos
  * treliça
  * portico
* Encontrar os esforços de cada barra
  * Ações de extremidade finais de cada barra




### Limitações:

- 2 elementos por barra apenas
- Elementos planos apenas
- Consideremos apenas apoios que restringem todo o deslocamento (ou rotação)